In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import KFold
from numpy import mean, argmax

import loadDataset
import models
import augment

c:\users\kyle\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X, Y = loadDataset.loadDataSet('./dataset/imgs/rock_frames', './dataset/imgs/paper_frames',
                               './dataset/imgs/scissor_frames', './dataset/csvs/rock.csv',
                               './dataset/csvs/paper.csv', './dataset/csvs/scissor.csv')
X = np.stack([cv2.Canny(X[i], 50, 100) for i in range(X.shape[0])])  # Apply Canny Edge Detection
X = np.expand_dims(X, 3)  # Make images 3D for tensorflow ops.

X1, Y1 = X, (Y > 0).astype(int)  # 0: None/NA, 1: Rock/Paper/Scissors
X2, Y2 = X[Y > 0], Y[Y > 0] - 1  # Y is 0-based containing only Rock, Paper, or Scissors.

In [3]:
video_len = 45

# Custom Batch Generator
def _batches_video(inputs, batch_size, shuffle=False, allow_smaller_final_batch=False):
    if not isinstance(inputs, list) and not isinstance(inputs, tuple):
        raise TypeError("Inputs must be of type list or tuple.")
    if not all([isinstance(x, np.ndarray) for x in inputs]):
        raise TypeError("Each input in the input list must be a numpy array.")
    total_size = inputs[0].shape[0]
    if not all([x.shape[0] == total_size for x in inputs]):
        raise RuntimeError("All inputs must have equal first dimension.")

    total_size -= video_len
    order = np.arange(total_size) if shuffle is False else np.random.permutation(total_size)
    for i in range(int(total_size / batch_size)):
        start_indices = order[(i)*batch_size:(i+1)*batch_size]                    
        yield [np.concatenate([x[None, (start):(start+video_len)] for start in start_indices]) for x in inputs]
    if allow_smaller_final_batch:
        start_indices = order[int(total_size / batch_size)*batch_size:]                    
        yield [np.concatenate([x[None, (start):(start+video_len)] for start in start_indices]) for x in inputs]
        
def _batches(inputs, batch_size, shuffle=False, allow_smaller_final_batch=False):
    if not isinstance(inputs, list) and not isinstance(inputs, tuple):
        raise TypeError("Inputs must be of type list or tuple.")
    if not all([isinstance(x, np.ndarray) for x in inputs]):
        raise TypeError("Each input in the input list must be a numpy array.")
    total_size = inputs[0].shape[0]
    if not all([x.shape[0] == total_size for x in inputs]):
        raise RuntimeError("All inputs must have equal first dimension.")

    order = np.arange(total_size) if shuffle is False else np.random.permutation(total_size)
    for i in range(int(total_size / batch_size)):
        yield [x[order[(i)*batch_size:(i+1)*batch_size]] for x in inputs]
    if allow_smaller_final_batch:
        yield [x[order[int(total_size / batch_size)*batch_size:]] for x in inputs]

### Fit Models

In [4]:
def fit1(session, model, x, y, batch_size=64, epochs=5, shuffle=False, verbose=True, print_interval=100):    
    [predict_op, loss_op, accuracy_op, train_op], (X, Y, training) = model
    augment_op = augment._augment_video_oper(img_size=[80, 80, 1], vid_length=45)
    
    session.run(tf.global_variables_initializer())
    session.run(tf.local_variables_initializer())
    
    train_loss, train_accuracy = [], []
    valid_loss, valid_accuracy = [], []
    
    num_train_batches = int(x.shape[0] / batch_size)
    
    # Training
    print("Beginning Training...")
    for e in range(epochs):        
        sum_loss, sum_accuracy = 0, 0
        for batch_i, (batch_x, batch_y) in enumerate(_batches_video([x, y], batch_size=batch_size, shuffle=shuffle, allow_smaller_final_batch=False)):
            batch_x = np.stack([augment.augment_video(batch_x[i], session, augment_op) for i in range(batch_size)])
            batch_y = batch_y[:, -1]
                
            loss, accuracy, _ = session.run([loss_op, accuracy_op, train_op],
                                            feed_dict={X: batch_x, Y: batch_y, training: True})

            if verbose and batch_i % print_interval == 0:
                print("Train Batch {}: Loss = {}, Accuracy = {}".format(batch_i, loss, accuracy))
            sum_loss += loss
            sum_accuracy += accuracy

        train_loss.append(sum_loss / num_train_batches)
        train_accuracy.append(sum_accuracy / num_train_batches)
        if verbose:
            print("Epoch {}: Average Train Loss = {}, Average Train Accuracy = {}\n"
                  .format(e + 1, train_loss[e], train_accuracy[e]))
    
    plt.figure()
    plt.title("Training Loss per Epoch")
    plt.plot(np.arange(epochs), np.array(train_loss), label="Training")
    plt.xlabel("Epoch")
    plt.xticks(np.arange(epochs), np.arange(epochs))
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
    
def fit2(session, model, x, y, batch_size=64, epochs=5, shuffle=False, verbose=True, print_interval=100):
    [predict_op, loss_op, accuracy_op, train_op], (X, Y, training) = model
    augment_op = augment._augment_video_oper(img_size=[80, 80, 1], vid_length=1)
    
    session.run(tf.global_variables_initializer())
    session.run(tf.local_variables_initializer())
    
    train_loss, train_accuracy = [], []
    valid_loss, valid_accuracy = [], []
    
    num_train_batches = int(x.shape[0] / batch_size)
    
    # Training
    print("Beginning Training...")
    for e in range(epochs):
        sum_loss, sum_accuracy = 0, 0
        for batch_i, (batch_x, batch_y) in enumerate(_batches([x, y], batch_size=batch_size, shuffle=shuffle, allow_smaller_final_batch=False)):
            batch_x = np.expand_dims(batch_x, 1)
            batch_x = np.stack([augment.augment_video(batch_x[i], session, augment_op) for i in range(batch_size)])
            batch_x = np.squeeze(batch_x, 1)
            
            loss, accuracy, _ = session.run([loss_op, accuracy_op, train_op],
                                            feed_dict={X: batch_x, Y: batch_y, training: True})

            if verbose and batch_i % print_interval == 0:
                print("Train Batch {}: Loss = {}, Accuracy = {}".format(batch_i, loss, accuracy))
            sum_loss += loss
            sum_accuracy += accuracy

        train_loss.append(sum_loss / num_train_batches)
        train_accuracy.append(sum_accuracy / num_train_batches)
        if verbose:
            print("Epoch {}: Average Train Loss = {}, Average Train Accuracy = {}\n"
                  .format(e + 1, train_loss[e], train_accuracy[e]))
    
    plt.figure()
    plt.title("Training Loss per Epoch")
    plt.plot(np.arange(epochs), np.array(train_loss), label="Training")
    plt.xlabel("Epoch")
    plt.xticks(np.arange(epochs), np.arange(epochs))
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

In [5]:
tf.reset_default_graph()
with tf.Session() as session:
    input_size = [64, 64, 1]
    model1 = models.model1(input_size, image_history_length=45)
    fit1(session, model1, X1, Y1, epochs=3, shuffle=True, verbose=True, print_interval=10)
    model2 = models.model2(input_size)
    fit2(session, model2, X2, Y2, epochs=3, shuffle=True, verbose=True, print_interval=10)
    
    # Save Model
    # https://www.tensorflow.org/programmers_guide/saved_model
    save_path = os.path.join(os.getcwd(), "savedmodels\\both\\models.ckpt")
    saver = tf.train.Saver()
    saver.save(session, save_path)
    print("Session Saved.")

TypeError: __int__ returned non-int (type NoneType)

### Cross-Validate Models

In [ ]:
# Custom Cross-Validation that uses Custom Batch Generator
def cross_validation1(session, model, x, y, batch_size=64, epochs=5, K=5, shuffle=False, verbose=True, print_interval=100):
    # https://stackoverflow.com/questions/39748660/how-to-perform-k-fold-cross-validation-with-tensorflow
    [predict_op, loss_op, accuracy_op, train_op], (X, Y, training) = model

    # K-Fold Loop
    train_loss, train_accuracy = [], []
    valid_loss, valid_accuracy = [], []
    k = 0
    for train_i, valid_i in KFold(n_splits=K).split(x):
        train_loss.append([])
        train_accuracy.append([])

        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())

        num_train_batches = int(x[train_i].shape[0] / batch_size)
        num_valid_batches = int(x[valid_i].shape[0] / batch_size)

        # Training
        for e in range(epochs):
            sum_loss, sum_accuracy = 0, 0
            for batch_i, (batch_x, batch_y) in enumerate(_batches_video([x[train_i], y[train_i]], batch_size=batch_size, shuffle=shuffle, allow_smaller_final_batch=False)):
                batch_y = batch_y[:, -1]
                loss, accuracy, _ = session.run([loss_op, accuracy_op, train_op],
                                                feed_dict={X: batch_x, Y: batch_y, training: True})

                if verbose and batch_i % print_interval == 0:
                    print("Train Batch {}: Loss = {}, Accuracy = {}".format(batch_i, loss, accuracy))
                sum_loss += loss
                sum_accuracy += accuracy

            train_loss[k].append(sum_loss / num_train_batches)
            train_accuracy[k].append(sum_accuracy / num_train_batches)
            if verbose:
                print("Epoch {}: Average Train Loss = {}, Average Train Accuracy = {}\n"
                      .format(e + 1, train_loss[k][e], train_accuracy[k][e]))

        # Validation
        sum_loss, sum_accuracy = 0, 0
        for batch_i, (batch_x, batch_y) in enumerate(_batches_video([x[valid_i], y[valid_i]], batch_size=batch_size, shuffle=shuffle, allow_smaller_final_batch=False)):
            batch_y = batch_y[:, -1]
            loss, accuracy = session.run([loss_op, accuracy_op],
                                         feed_dict={X: batch_x, Y: batch_y, training: False})

            if verbose and batch_i % print_interval == 0:
                print("Valid Batch {}: Loss = {}, Accuracy = {}".format(batch_i, loss, accuracy))
            sum_loss += loss
            sum_accuracy += accuracy

        valid_loss.append(sum_loss / num_valid_batches)
        valid_accuracy.append(sum_accuracy / num_valid_batches)
        if verbose:
            print("Fold {}: Validation Loss = {}, Validation Accuracy = {}\n"
                  .format(k + 1, valid_loss[k], valid_accuracy[k]))

        k += 1

    # Results
    print("Average Valid Loss = {}, Average Valid Accuracy = {}".format(mean(valid_loss), mean(valid_accuracy)))

    plt.figure()
    plt.title("Training Loss per Epoch")
    plt.plot(np.arange(epochs), np.array(train_loss).T)
    plt.xlabel("Epoch")
    plt.xticks(np.arange(epochs), np.arange(epochs))
    plt.ylabel("Loss")
    plt.legend(["Fold %d" % i for i in range(1, K+1)])
    plt.show()
    
def cross_valididation2(session, model, x, y, batch_size=64, epochs=5, K=5, shuffle=False, verbose=True, print_interval=100):
    # https://stackoverflow.com/questions/39748660/how-to-perform-k-fold-cross-validation-with-tensorflow
    [predict_op, loss_op, accuracy_op, train_op], (X, Y, training) = model

    # K-Fold Loop
    train_loss, train_accuracy = [], []
    valid_loss, valid_accuracy = [], []
    k = 0
    for train_i, valid_i in KFold(n_splits=K).split(x):
        train_loss.append([])
        train_accuracy.append([])

        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())

        num_train_batches = int(x[train_i].shape[0] / batch_size)
        num_valid_batches = int(x[valid_i].shape[0] / batch_size)

        # Training
        for e in range(epochs):
            sum_loss, sum_accuracy = 0, 0
            for batch_i, (batch_x, batch_y) in enumerate(_batches([x[train_i], y[train_i]], batch_size=batch_size, shuffle=shuffle, allow_smaller_final_batch=False)):
                loss, accuracy, _ = session.run([loss_op, accuracy_op, train_op],
                                                feed_dict={X: batch_x, Y: batch_y, training: True})

                if verbose and batch_i % print_interval == 0:
                    print("Train Batch {}: Loss = {}, Accuracy = {}".format(batch_i + 1, loss, accuracy))
                sum_loss += loss
                sum_accuracy += accuracy

            train_loss[k].append(sum_loss / num_train_batches)
            train_accuracy[k].append(sum_accuracy / num_train_batches)
            if verbose:
                print("Epoch {}: Average Train Loss = {}, Average Train Accuracy = {}\n"
                      .format(e + 1, train_loss[k][e], train_accuracy[k][e]))

        # Validation
        sum_loss, sum_accuracy = 0, 0
        for batch_i, (batch_x, batch_y) in enumerate(_batches([x[valid_i], y[valid_i]], batch_size=batch_size, shuffle=shuffle, allow_smaller_final_batch=False)):
            loss, accuracy = session.run([loss_op, accuracy_op],
                                         feed_dict={X: batch_x, Y: batch_y, training: False})

            if verbose and batch_i % print_interval == 0:
                print("Valid Batch {}: Loss = {}, Accuracy = {}".format(batch_i + 1, loss, accuracy))
            sum_loss += loss
            sum_accuracy += accuracy

        valid_loss.append(sum_loss / num_valid_batches)
        valid_accuracy.append(sum_accuracy / num_valid_batches)
        if verbose:
            print("Fold {}: Validation Loss = {}, Validation Accuracy = {}\n"
                  .format(k + 1, valid_loss[k], valid_accuracy[k]))

        k += 1

    # Results
    print("Average Valid Loss = {}, Average Valid Accuracy = {}".format(mean(valid_loss), mean(valid_accuracy)))

    plt.figure()
    plt.title("Training Loss per Epoch")
    plt.plot(np.arange(epochs), np.array(train_loss).T)
    plt.xlabel("Epoch")
    plt.xticks(np.arange(epochs), np.arange(epochs))
    plt.ylabel("Loss")
    plt.legend(["Fold %d" % i for i in range(1, K+1)])
    plt.show()

In [ ]:
tf.reset_default_graphs()
with tf.Session() as session:
    image_shape = X1.shape[1:]
    model = models.model1(image_shape)
    cross_validation1(model1, session, X1, Y1, epochs=5, shuffle=True, print_interval=10)

In [ ]:
tf.reset_default_graphs()
with tf.Session() as session:
    image_shape = X2.shape[1:]
    mode2 = models.model2(image_shape)
    cross_validation2(model2, session, X2, Y2, epochs=5, shuffle=True, print_interval=10)